Playground for FI functions for Wang-Buszaki and Traub+Ca. We already have the data, but we just need a convenient way to play with the data (e.g., get/use the inverse functions).

In [19]:
import numpy as np
import matplotlib.pylab as mp
from scipy.interpolate import interp1d


# load data
ftb_data = np.loadtxt('tbfi2.dat')
ftb_tab = np.loadtxt('tbfi2.tab')
fwb_data = np.loadtxt('wbfi.tab')

# name some values for easy reading
ftb_input_lower_bound = ftb_data[0,0]
ftb_input_upper_bound = ftb_data[-1,0]

fwb_input_lower_bound = fwb_data[1]
fwb_input_upper_bound = fwb_data[2]

fwb_domain = np.linspace(fwb_input_lower_bound,fwb_input_upper_bound,fwb_data[0])
ftb_domain = np.linspace(ftb_tab[1],ftb_tab[2],ftb_tab[0])

# define interpolating functions for frequency-current
tbfi = interp1d(ftb_data[:,0],ftb_data[:,1])
wbfi = interp1d(fwb_domain,fwb_data[3:])

# define inverse interpolating functions for frequency-current
# well-defind because the FI curves are monotonic
tbfi_inv = interp1d(ftb_data[:,1],ftb_data[:,0])
wbfi_inv = interp1d(fwb_data[3:],fwb_domain)

# determine polynomial interpolation of tb
p_coeffs = np.polyfit(ftb_domain,ftb_tab[3:],15)
totsol = 0
totstr = ''
for i in range(len(p_coeffs)):
    totsol += p_coeffs[::-1][i]*(ftb_domain**i)
    
    if p_coeffs[::-1][i] > 0:
        totstr += '+'+str(p_coeffs[::-1][i])+'*x^'+str(i)
    else:
        totstr += '-'+str(np.abs(p_coeffs[::-1][i]))+'*x^'+str(i)

print totstr
print len(ftb_domain),len(totsol)
mp.figure()
mp.plot(ftb_domain,ftb_tab[3:])
mp.plot(ftb_domain,totsol)
mp.show()

# determine polynomial interpolation of wb
print len(fwb_domain),len(fwb_data[3:])
p_coeffs = np.polyfit(fwb_domain,fwb_data[3:],15)
totsol = 0
totstr = ''
for i in range(len(p_coeffs)):
    totsol += p_coeffs[::-1][i]*(fwb_domain**i)
    
    if p_coeffs[::-1][i] > 0:
        totstr += '+'+str(p_coeffs[::-1][i])+'*x^'+str(i)
    else:
        totstr += '-'+str(np.abs(p_coeffs[::-1][i]))+'*x^'+str(i)

print totstr

mp.figure()
mp.plot(fwb_domain,fwb_data[3:])
mp.plot(fwb_domain,totsol)
mp.show()



# based on simulations, the maximum change in the period of the models
# is from 20 to around 22 or 23 (or 18,17 min).
# I.e. the freq changes from 0.05 to 0.04545... or 0.0434782609
# (the max freq is 1/18=.055..., 1/17=.0588235294)
# we find the input current values at these extrema, and determine the discretization
# size and step.

min_f = 0.0434782609
max_f = 0.0588235294

print 'input current values', tbfi_inv(.05),wbfi_inv(.05)

# find min current, tb
tb_min_I = tbfi_inv(min_f)
# find max current, tb
tb_max_I = tbfi_inv(max_f)

# find min current, wb
wb_min_I = wbfi_inv(min_f)
# find max current, wb
wb_max_I = wbfi_inv(max_f)

# round off to nice decimal place
tb_min_I=np.round(tb_min_I,2);tb_max_I=np.round(tb_max_I,2)
wb_min_I=np.round(wb_min_I,2);wb_max_I=np.round(wb_max_I,2)

print tb_min_I,tb_max_I
print wb_min_I,wb_max_I

# determine approximate discretization size for 20 intervals
print (tb_max_I - tb_min_I)/20
print (wb_max_I - wb_min_I)/20

# show period
print 1/tbfi(tb_min_I),1/tbfi(tb_max_I)
print 1/wbfi(wb_min_I),1/wbfi(wb_max_I)


-0.0347852888226*x^0+0.186873169095*x^1-0.409677440916*x^2+0.54289364772*x^3-0.468257572487*x^4+0.279190605143*x^5-0.119326576663*x^6+0.0373899858244*x^7-0.00869727639496*x^8+0.00150788546107*x^9-0.000193974777049*x^10+1.82423993796e-05*x^11-1.21770558233e-06*x^12+5.4610481018e-08*x^13-1.47481489996e-09*x^14+1.8118842417e-11*x^15
10000 10000
1000 1000
-0.508521266473*x^0+11.4772506459*x^1-116.083294553*x^2+701.459125254*x^3-2807.76852323*x^4+7890.50478887*x^5-16113.4348453*x^6+24406.8284532*x^7-27715.558749*x^8+23654.0088034*x^9-15084.462889*x^10+7074.60540046*x^11-2367.01538907*x^12+534.389241681*x^13-72.9226220434*x^14+4.54148967685*x^15
input current values 6.0416 0.809079373711
5.41 6.86
0.69 0.98
0.0725
0.0145
22.9752319781 17.0098027734
22.9814163806 17.0299734709


The data files are generated using xpp:

        Step 1. Compute a singel orbit, adjoint, and H function (to load the 
        program with the correct right-hand sides for H function. Or just load in
        set file where it was done
        Step 2.  Set transient to some reasonable number to assure convergence 
        onto the limit cycle as you change parameters and total to be at least
        2 periods beyond the transient
        Step 3. Se up Poincare map - period - stop on section. This lets you
        get the period
        Step 4. In numerics averaging - click on adjrange
        Step 5. Initconds range over the parameter. It should find the periodic
        orbit, adjoint, and H function and save. Files are of the form
        orbit.parname_parvalue.dat etc

Using this method, we generate a bunch of dat files (adjoint.\*, hfun.\*, orbit.\*). However, we only need the orbit data since we only want limit cycle lookup tables. So we delete the adjoint and hfun data and we are left with data files named "orbit.i\_\*.dat". We convert these data files into tab files below.

In [2]:
import numpy as np


tb_var_names = ['vt','mt','nt','ht','wt','st','ca']
wb_var_names = ['v','h','n']

tb_valid_currents = np.array([ 5.41,5.4825,5.555,5.6275,5.7,5.7725,5.845,
                                5.9175,5.99, 6.0625,6.135,6.2075,6.28,6.3525,
                                6.425,6.4975,6.57,6.6425,6.715,6.7875])

wb_valid_currents = np.array([0.69  ,  0.7045,  0.719 ,  0.7335,  0.748 ,  0.7625,  0.777 ,
                                0.7915,  0.806 ,  0.8205,  0.835 ,  0.8495,  0.864 ,  0.8785,
                                0.893 ,  0.9075,  0.922 ,  0.9365,  0.951 ,  0.9655])


if False:
    nskip = 20
    for i in range(len(tb_valid_currents)):
        current = tb_valid_currents[i]
        data = np.loadtxt('orbit.i_'+str(current)+'.dat')[::nskip,:]

        tlo = data[0,0]
        thi = data[-1,0]
        for j in range(len(tb_var_names)):
        
            tabledata = data[:,j+1]
            N = len(tabledata)
        
            tabledata = np.insert(tabledata,0,thi)
            tabledata = np.insert(tabledata,0,tlo)
            tabledata = np.insert(tabledata,0,N)
        
            var = tb_var_names[j]
            tablename = var+'_i'+str(current)+'.tab'
        
            # if dir tabular does not exist, create it.
            np.savetxt('tabular/'+tablename,tabledata)
    
    
    
nskip = 10
for i in range(len(wb_valid_currents)):
    current = wb_valid_currents[i]
    data = np.loadtxt('orbit.i0_'+str(current)+'.dat')[::nskip,:]

    tlo = data[0,0]
    thi = data[-1,0]
    for j in range(len(wb_var_names)):
        
        tabledata = data[:,j+1]
        N = len(tabledata)
        
        tabledata = np.insert(tabledata,0,thi)
        tabledata = np.insert(tabledata,0,tlo)
        tabledata = np.insert(tabledata,0,N)
        
        var = wb_var_names[j]
        tablename = var+'_i0'+str(current)+'.tab'
        
        # if dir tabular does not exist, create it.
        np.savetxt('tabular/'+tablename,tabledata)
    